# Importaciones

In [1]:
from datetime import datetime
from datetime import timedelta
import concurrent.futures
from pytz import timezone

import spacy
import pytextrank
import psycopg
import newspaper

from extract_news import news_extractor_per_media, news_extractor_per_media_concurrent
from medias.medias_list import medias_dict

c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages


In [2]:
nlp = spacy.load('es_core_news_md')

# Notas

* Documentación de [psycopg3](https://www.psycopg.org/psycopg3/docs/basic/index.html)
* Documentación de [Newspaper4k](https://pypi.org/project/newspaper4k/)
* *connections string*: https://www.postgresql.org/docs/current/libpq-connect.html#LIBPQ-CONNSTRING
* [Uso de *threts* en Pyscopg 3](https://www.psycopg.org/psycopg3/docs/advanced/async.html)
* [Uso de parámetros en consultas](https://www.psycopg.org/psycopg3/docs/basic/params.html)

<br>

## *Cron jobs* en Railway:
* [Cron Jobs](https://docs.railway.app/reference/cron-jobs)
* [Running a Scheduled Job](https://docs.railway.app/guides/cron-jobs)
* [Ejemplo en JavaScript](https://blog.railway.app/p/cron-jobs)

Error al usar la función `build` de *newspaper*:

* [AttributeError: 'NoneType' object has no attribute 'xpath'](https://github.com/codelucas/newspaper/issues/646)

# Base de datos

In [4]:
conn_postgresql = psycopg.connect(
    host="localhost",
    dbname="news_kg_v1",
    user="postgres",
    password="0rioN-689"
)

In [ ]:
conn_postgresql.execute(
'''
CREATE TABLE IF NOT EXISTS news_chile (
  id SERIAL PRIMARY KEY,
  media_name VARCHAR(150) NULL,
  url TEXT NULL,
  date TIMESTAMP NULL,
  author VARCHAR(150) NULL,
  body TEXT NULL,
  keywords TEXT NULL
);
'''
)

In [ ]:
conn_postgresql.commit()

In [ ]:
conn_postgresql.close()

## Populando base de datos

In [3]:
today = datetime.today()
tz = timezone('UTC')
today = today.replace(tzinfo=tz)
time_delta = timedelta(days=1, hours=today.hour, minutes=today.minute)

In [4]:
news_extractor_per_media(
    country='chile',
    medias=medias_dict,
    today=today,
    t_delta=time_delta,
    conn=conn_postgresql
)

c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages\newspaper\source.py:260: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if feed.doc:


Ha ocurrido un error con https://www.aricamia.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://elreporterodeiquique.com/:
Article is binary data: https://elreporterodeiquique.com/category/deportes/feed
Ha ocurrido un error con https://ovallehoy.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://www.observador.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://www.elinformador.cl/:
'NoneType' object has no attribute 'xpath'


c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


Ha ocurrido un error con https://www.laizquierdadiario.cl/Chile:
Http error when downloading https://www.laizquierdadiario.cl/Chile. Status code: {status_code}
Ha ocurrido un error con https://www.biobiochile.cl/:
Article is binary data: https://www.biobiochile.cl/frecuencias_rbb.pdf
Ha ocurrido un error con https://www.elperiscopio.cl/:
'NoneType' object has no attribute 'xpath'
Ha ocurrido un error con https://www.theclinic.cl/:
Http error when downloading https://www.theclinic.cl/. Status code: {status_code}


In [4]:
with concurrent.futures.ThreadPoolExecutor() as tpe:
    results = [tpe.submit(news_extractor_per_media_concurrent, country='chile', media=media, today=today, t_delta=time_delta, conn=conn_postgresql) for media in medias_dict['chile']]

Ha ocurrido un error con https://www.aricamia.cl/:
'NoneType' object has no attribute 'xpath'
Concluida extracción de noticias de https://www.aricamia.cl/


c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages\newspaper\source.py:260: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if feed.doc:


Concluida extracción de noticias de https://www.chasquis.cl/diario/
Concluida extracción de noticias de https://lakalle.cl/
Concluida extracción de noticias de https://www.aricahoy.cl/
Concluida extracción de noticias de https://www.elnortero.cl/
Concluida extracción de noticias de https://www.elmorrocotudo.cl/
Concluida extracción de noticias de https://www.elboyaldia.cl/
Concluida extracción de noticias de https://www.diarioantofagasta.cl/
Ha ocurrido un error con https://elreporterodeiquique.com/:
Article is binary data: https://elreporterodeiquique.com/category/deportes/rss
Concluida extracción de noticias de https://elreporterodeiquique.com/
Concluida extracción de noticias de https://www.aricaldia.cl/
Concluida extracción de noticias de https://www.elquehaydecierto.cl/
Concluida extracción de noticias de https://www.atacamaenlinea.cl/
Concluida extracción de noticias de http://www.elovallino.cl/
Concluida extracción de noticias de https://www.diarioeldia.cl/
Concluida extracción 

c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
c:\Users\leona\.virtualenvs\proyecto_kg_news-rvs_gk5n\lib\site-packages\PIL\TiffImagePlugin.py:870: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Concluida extracción de noticias de https://elhuemul.cl/
Concluida extracción de noticias de http://web.elpatagondomingo.cl/
Concluida extracción de noticias de https://chile.as.com/
Concluida extracción de noticias de https://www.diarioviregion.cl/


# Consultas

In [4]:
q = '''
SELECT id, keywords FROM news_chile LIMIT 50
'''
cursor = conn_postgresql.cursor()
cursor.execute(q)
cursor.fetchall()

[(1, None),
 (2, None),
 (3, None),
 (4, None),
 (5, None),
 (6, None),
 (7, None),
 (8, None),
 (47, None),
 (48, None),
 (9, None),
 (10, None),
 (11, None),
 (12, None),
 (13, None),
 (14, None),
 (15, None),
 (16, None),
 (17, None),
 (18, None),
 (19, None),
 (575, None),
 (576, None),
 (577, None),
 (578, None),
 (20, None),
 (21, None),
 (22, None),
 (23, None),
 (24, None),
 (25, None),
 (26, None),
 (27, None),
 (28, None),
 (655, None),
 (29, None),
 (30, None),
 (31, None),
 (32, None),
 (33, None),
 (34, None),
 (35, None),
 (36, None),
 (61, None),
 (37, None),
 (38, None),
 (39, None),
 (40, None),
 (77, None),
 (656, None)]

# *textrank*

In [5]:
nlp.add_pipe("textrank")

ValueError: [E007] 'textrank' already exists in pipeline. Existing names: ['tok2vec', 'morphologizer', 'parser', 'senter', 'attribute_ruler', 'lemmatizer', 'ner', 'textrank']

In [7]:
q = '''
SELECT id, body FROM news_chile LIMIT 50
'''
cursor = conn_postgresql.cursor()
cursor.execute(q)
example_news = cursor.fetchall()

In [27]:
doc = nlp(example_news[15][1])

In [28]:
for phrase in doc._.phrases:
    print(f"text: {phrase.text}\trank: {phrase.rank}\tcount: {phrase.count}")

text: Partido Republicano	rank: 0.07387075734357695	count: 8
text: Chile Vamos	rank: 0.07072913315404325	count: 3
text: Atacama	rank: 0.06212819319225798	count: 12
text: fuerza	rank: 0.06133806015187843	count: 1
text: Chile	rank: 0.06077033490871698	count: 6
text: Meza Pereira	rank: 0.053319621568505046	count: 2
text: José Carlos Meza Pereira	rank: 0.05281463537715464	count: 2
text: Gobierno Regional	rank: 0.05118925184548504	count: 1
text: trabajo	rank: 0.04603686605089563	count: 1
text: Presidente Arturo Squella	rank: 0.04407530696170517	count: 1
text: Gobernación Regional	rank: 0.043830450703106934	count: 1
text: distrito N	rank: 0.04246124685825625	count: 1
text: Meza	rank: 0.04114013961925024	count: 2
text: conversaciones	rank: 0.03994705731297133	count: 1
text: Gobierno	rank: 0.039447237813972524	count: 1
text: el Partido Republicano	rank: 0.03917588508677192	count: 1
text: Coquimbo	rank: 0.03909132241567253	count: 2
text: Republicanos	rank: 0.037618919622052195	count: 4
text: De

In [24]:
doc = nlp(example_news[10][1])
for phrase in doc._.phrases:
    if phrase.text in [token for token in doc if token.is_stop]:
        print("keyword is stop")
        #print(f"text: }\trank: {phrase.rank}\tcount: {phrase.count}")

In [26]:
doc = nlp(example_news[10][1])
for phrase in doc._.phrases:
    print(type(phrase.text))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class